In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
import arviz as az
from scipy.special import softmax
import warnings
warnings.filterwarnings('ignore')

# 模型结果分析

In [2]:
# 读取结果
trace = az.from_netcdf('maas_hmm_results/trace.nc')

# 查看摘要统计
print(az.summary(trace))


                  mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
alpha_init[0]    1.616  2.874  -4.060    5.417      1.487    0.585       5.0   
alpha_init[1]   -0.309  2.755  -4.621    4.748      1.398    0.451       4.0   
beta_init[0, 0]  0.041  0.585  -1.082    1.113      0.018    0.019     998.0   
beta_init[0, 1]  0.068  0.654  -1.161    1.317      0.025    0.018     683.0   
beta_init[1, 0] -0.206  0.623  -1.366    0.961      0.028    0.023     506.0   
...                ...    ...     ...      ...        ...      ...       ...   
gamma_trans[2]   0.069  0.267  -0.408    0.614      0.059    0.013      18.0   
gamma_trans[3]  -0.016  0.362  -0.643    0.702      0.111    0.012       9.0   
pi_init_mean[0]  0.291  0.110   0.088    0.509      0.012    0.009      90.0   
pi_init_mean[1]  0.446  0.176   0.098    0.667      0.098    0.043       4.0   
pi_init_mean[2]  0.264  0.189   0.000    0.598      0.099    0.048       5.0   

                 ess_tail  r_hat  
alph

In [3]:
trace.posterior

<xarray.Dataset> Size: 1MB
Dimensions:                 (chain: 2, draw: 1000, alpha_init_dim_0: 2,
                             beta_init_dim_0: 11, beta_init_dim_1: 2,
                             ASC_t1_dim_0: 3, ASC_t1_dim_1: 4,
                             beta_time_t1_dim_0: 3, beta_rail_t1_dim_0: 3,
                             ASC_t2_dim_0: 3, ASC_t2_dim_1: 4,
                             beta_price_t2_dim_0: 3, trans_logits_raw_dim_0: 3,
                             trans_logits_raw_dim_1: 2, gamma_trans_dim_0: 4,
                             pi_init_mean_dim_0: 3)
Coordinates: (12/16)
  * chain                   (chain) int64 16B 0 1
  * draw                    (draw) int64 8kB 0 1 2 3 4 5 ... 995 996 997 998 999
  * alpha_init_dim_0        (alpha_init_dim_0) int64 16B 0 1
  * beta_init_dim_0         (beta_init_dim_0) int64 88B 0 1 2 3 4 5 6 7 8 9 10
  * beta_init_dim_1         (beta_init_dim_1) int64 16B 0 1
  * ASC_t1_dim_0            (ASC_t1_dim_0) int64 24B 0 1 2
    ...                      ...
  * ASC_t2_dim_1            (ASC_t2_dim_1) int64 32B 0 1 2 3
  * beta_price_t2_dim_0     (beta_price_t2_dim_0) int64 24B 0 1 2
  * trans_logits_raw_dim_0  (trans_logits_raw_dim_0) int64 24B 0 1 2
  * trans_logits_raw_dim_1  (trans_logits_raw_dim_1) int64 16B 0 1
  * gamma_trans_dim_0       (gamma_trans_dim_0) int64 32B 0 1 2 3
  * pi_init_mean_dim_0      (pi_init_mean_dim_0) int64 24B 0 1 2
Data variables:
    alpha_init              (chain, draw, alpha_init_dim_0) float64 32kB 3.49...
    beta_init               (chain, draw, beta_init_dim_0, beta_init_dim_1) float64 352kB ...
    ASC_t1                  (chain, draw, ASC_t1_dim_0, ASC_t1_dim_1) float64 192kB ...
    beta_time_t1            (chain, draw, beta_time_t1_dim_0) float64 48kB -0...
    beta_rail_t1            (chain, draw, beta_rail_t1_dim_0) float64 48kB -0...
    ASC_t2                  (chain, draw, ASC_t2_dim_0, ASC_t2_dim_1) float64 192kB ...
    beta_price_t2           (chain, draw, beta_price_t2_dim_0) float64 48kB -...
    trans_logits_raw        (chain, draw, trans_logits_raw_dim_0, trans_logits_raw_dim_1) float64 96kB ...
    gamma_trans             (chain, draw, gamma_trans_dim_0) float64 64kB -0....
    pi_init_mean            (chain, draw, pi_init_mean_dim_0) float64 48kB 0....
Attributes:
    created_at:                 2025-12-28T07:23:59.235470+00:00
    arviz_version:              0.23.0
    inference_library:          pymc
    inference_library_version:  5.27.0
    sampling_time:              171.28537440299988
    tuning_steps:               500

In [4]:
def load_data(filepath):
    """
    加载合并后的问卷数据
    
    Parameters
    ----------
    filepath : str
        数据文件路径
    """
    # 根据文件类型选择读取方式
    if filepath.endswith('.csv'):
        df = pd.read_csv(filepath, encoding='gb2312')
    elif filepath.endswith('.xlsx'):
        df = pd.read_excel(filepath)
    else:
        raise ValueError("不支持的文件格式")
    
    return df

In [5]:
# 使用模拟数据测试 (实际使用时替换为真实数据)
data = load_data('data/最终模型数据.csv')

In [6]:
data['maas'].value_counts()

maas
1    15620
4     4090
5     3545
6     2325
7     1780
Name: count, dtype: int64

# 态度转移因素

## 第一阶段态度

In [7]:
data['choose_options'] = data['maas'].apply(lambda x: 1 if x >= 4 else 0)

In [8]:
data['time_savings'] = 0

In [9]:
data.loc[data['maas'] == 4, 'time_savings'] = data.loc[data['maas'] == 4]['M1dif_triptime']
data.loc[data['maas'] == 5, 'time_savings'] = data.loc[data['maas'] == 5]['M2dif_triptime']
data.loc[data['maas'] == 6, 'time_savings'] = data.loc[data['maas'] == 6]['M3dif_triptime']
data.loc[data['maas'] == 7 ,'time_savings'] = data.loc[data['maas'] == 7]['M4dif_triptime']

In [10]:
data['cost_savings'] = 0

In [11]:
data.loc[data['maas'] == 4, 'cost_savings'] = data.loc[data['maas'] == 4]['M1dif_price']
data.loc[data['maas'] == 5, 'cost_savings'] = data.loc[data['maas'] == 5]['M2dif_price']
data.loc[data['maas'] == 6, 'cost_savings'] = data.loc[data['maas'] == 6]['M3dif_price']
data.loc[data['maas'] == 7 ,'cost_savings'] = data.loc[data['maas'] == 7]['M4dif_price']

## 套餐匹配因素

In [12]:
data['week_ebike']

0        2
1        2
2        2
3        2
4        2
        ..
27355    1
27356    1
27357    1
27358    1
27359    1
Name: week_ebike, Length: 27360, dtype: int64

In [13]:
data['match_bus'] = data['week_bus'] > 3
data['match_metro'] = data['week_metro'] > 3
data['match_bike'] = data['week_bike'] > 3

In [14]:
data.loc[data['results'] == 4]['week_ebike'] * 4 + 1.5 - data.loc[data['results'] == 4]['ebike_4'] * 10

3       -22.5
8       -22.5
13      -22.5
61      -18.5
66      -18.5
         ... 
27195   -10.5
27196   -14.5
27197   -18.5
27198   -22.5
27199   -22.5
Length: 2258, dtype: float64

In [15]:
data['match_e_bike'] = 0
data.loc[data['results'] == 1, 'match_e_bike'] = data.loc[data['results'] == 1]['week_ebike'] * 4 + 1.5 - data.loc[data['results'] == 1]['ebike_12'] * 10
data.loc[data['results'] == 2, 'match_e_bike'] = data.loc[data['results'] == 2]['week_ebike'] * 4 + 1.5 - data.loc[data['results'] == 2]['ebike_12'] * 10
data.loc[data['results'] == 3, 'match_e_bike'] = data.loc[data['results'] == 3]['week_ebike'] * 4 + 1.5 - data.loc[data['results'] == 3]['ebike_3'] * 10
data.loc[data['results'] == 4 ,'match_e_bike'] = data.loc[data['results'] == 4]['week_ebike'] * 4 + 1.5 - data.loc[data['results'] == 4]['ebike_4'] * 10

In [16]:
## 居民出行调查是13.48公里

In [17]:
data['match_taxi'] = 0
data.loc[data['results'] == 1, 'match_taxi'] = ((data.loc[data['results'] == 1]['week_taxi'] * 4 + 1.5)*13.48 - data.loc[data['results'] == 1]['taxi_12'] * 100)/100
data.loc[data['results'] == 2, 'match_taxi'] = ((data.loc[data['results'] == 2]['week_taxi'] * 4 + 1.5)*13.48 - data.loc[data['results'] == 2]['taxi_12'] * 100)/100
data.loc[data['results'] == 3, 'match_taxi'] = ((data.loc[data['results'] == 3]['week_taxi'] * 4 + 1.5)*13.48 - data.loc[data['results'] == 3]['taxi_3'] * 100)/100
data.loc[data['results'] == 4 ,'match_taxi'] = ((data.loc[data['results'] == 4]['week_taxi'] * 4 + 1.5)*13.48 - data.loc[data['results'] == 4]['taxi_4'] * 100)/100

In [18]:
data

,maas,peopleID,purpose,distance,B1wtimeTaxi,A1ttimeCar,A1priceCar,B1ttimeTaxi,B1wtimeTaxi.1,B1triptime,...,age3,age4,choose_options,time_savings,cost_savings,match_bus,match_metro,match_bike,match_e_bike,match_taxi
0,1,9,0,9.1,10,1000,1000,18,10,28,...,False,False,0,0.0,0.0,True,True,False,9.5,1.8198
1,1,9,0,9.1,10,1000,1000,18,10,28,...,False,False,0,0.0,0.0,True,True,False,5.5,1.1198
2,1,9,0,9.1,10,1000,1000,18,10,28,...,False,False,0,0.0,0.0,True,True,False,-6.5,-2.3802
3,1,9,0,9.1,10,1000,1000,18,10,28,...,False,False,0,0.0,0.0,True,True,False,-22.5,-5.1302
4,1,9,0,9.1,10,1000,1000,18,10,28,...,False,False,0,0.0,0.0,True,True,False,1.5,0.4198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27355,1,2742,0,26.0,3,1000,1000,33,3,36,...,False,False,0,0.0,0.0,False,False,False,0.0,0.0000
27356,1,2742,0,26.0,3,1000,1000,33,3,36,...,False,False,0,0.0,0.0,False,False,False,0.0,0.0000
27357,1,2742,0,26.0,3,1000,1000,33,3,36,...,False,False,0,0.0,0.0,False,False,False,0.0,0.0000
27358,1,2742,0,26.0,3,1000,1000,33,3,36,...,False,False,0,0.0,0.0,False,False,False,0.0,0.0000


## 价格感知因素

In [19]:
cost_map = {
    1: 25,
    2: 100,
    3: 275,
    4: 600,
    5: 1150,
    6: 1500
}

In [20]:
data['cost']

0        3
1        3
2        3
3        3
4        3
        ..
27355    1
27356    1
27357    1
27358    1
27359    1
Name: cost, Length: 27360, dtype: int64

In [21]:
data['match_price'] = 0
data.loc[data['results'] == 1, 'match_price'] = (data.loc[data['results'] == 1]['cost'].map(cost_map))*0.01 - data.loc[data['results'] == 1]['price1']
data.loc[data['results'] == 2, 'match_price'] = (data.loc[data['results'] == 2]['cost'].map(cost_map))*0.01 - data.loc[data['results'] == 2]['price2']
data.loc[data['results'] == 3, 'match_price'] = (data.loc[data['results'] == 3]['cost'].map(cost_map))*0.01 - data.loc[data['results'] == 3]['price3']
data.loc[data['results'] == 4 ,'match_price'] = (data.loc[data['results'] == 4]['cost'].map(cost_map))*0.01 - data.loc[data['results'] == 4]['price4']

In [22]:
data['match_price'].describe()

count    27360.000000
mean        -1.896997
std          5.487067
min        -25.449200
25%         -3.386800
50%          0.000000
75%          0.000000
max         14.360000
Name: match_price, dtype: float64

In [31]:
data['price_ratio'] = 0
data.loc[data['results'] == 1, 'price_ratio'] = data.loc[data['results'] == 1]['price_12']
data.loc[data['results'] == 2, 'price_ratio'] = data.loc[data['results'] == 2]['price_12']
data.loc[data['results'] == 3, 'price_ratio'] = data.loc[data['results'] == 3]['price_3']
data.loc[data['results'] == 4 ,'price_ratio'] = data.loc[data['results'] == 4]['price_4']

In [35]:
data.to_csv('data/最终模型数据.csv', index=False)

In [34]:
data.isna().sum()

maas            0
peopleID        0
purpose         0
distance        0
B1wtimeTaxi     0
               ..
match_bike      0
match_e_bike    0
match_taxi      0
match_price     0
price_ratio     0
Length: 144, dtype: int64